
## 03 Feature Selection
**Objetivo**: Seleccionar las características más relevantes para el modelo.
**Contenido**:
- Técnicas de selección de características (e.g., selección univariante, selección recursiva, modelos basados en árboles).
- Evaluación de la importancia de las características.

In [1]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 663.4 kB/s eta 0:00:00


In [2]:
## Importar librerrias
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

In [3]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'

Mounted at /content/drive


In [4]:
seed = 42
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv')
df_identity_train = pd.read_csv(data_dir + '/train_identity.csv')
data = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left').sample(frac=0.1, random_state=seed)


In [5]:
dataset = data.copy()

In [6]:


# Suponiendo que ya tienes cargado tu dataset en 'dataset'

# Identificar tipos de datos
data_types = dataset.drop('isFraud',axis=1).dtypes

# Filtrar y contar variables numéricas y categóricas
num_vars = data_types[data_types != 'object']  # Variables numéricas
cat_vars = data_types[data_types == 'object']  # Variables categóricas

# Obtener nombres de las variables
num_vars_names = num_vars.index.tolist()
cat_vars_names = cat_vars.index.tolist()

# Mostrar resultados
print(f"Número de variables numéricas: {num_vars.shape[0]}")
print(f"Nombres de variables numéricas: {num_vars_names}")
print()
print(f"Número de variables categóricas: {cat_vars.shape[0]}")
print(f"Nombres de variables categóricas: {cat_vars_names}")


Número de variables numéricas: 402
Nombres de variables numéricas: ['TransactionID', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V

## Valores Nulos

In [7]:
# Calcula el porcentaje de valores nulos para cada columna
null_percentages = (dataset.isnull().sum() / len(dataset)) * 100

# Define los rangos para los valores nulos
ranges = [
    (0, 10),
    (10, 20),
    (20, 30),
    (30, 40),
    (40, 50),
    (50, 60),
    (60, 70),
    (70, 80),
    (80, 90),
    (90, 100)
]

# Agrupa las columnas en los rangos definidosa
grouped_columns = {f'{r[0]}-{r[1]}%': [] for r in ranges}

for column, percentage in null_percentages.items():
    for r in ranges:
        if r[0] <= percentage < r[1]:
            grouped_columns[f'{r[0]}-{r[1]}%'].append(column)

# Imprime los resultados
for range_, columns in grouped_columns.items():
    if columns:
        print(f'Rango de valores nulos: {range_}')
        print(f'Columnas: {columns}')
        print()

Rango de valores nulos: 0-10%
Columnas: ['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321']

Rango de valores nulos: 10-20%
Columnas: ['addr1',

- se observa los rangos porcentual de valores nulos en las columnas, continuaremos solo con las columnas menor a un 50% de valores nulos.

In [8]:
# Calcula el porcentaje de valores nulos para cada columna
null_percentages = (dataset.isnull().sum() / len(dataset)) * 100

# Define los rangos para los valores nulos
ranges = [
    (0, 50),
    (50, 100)
]

# Agrupa las columnas en los rangos definidosa
grouped_columns = {f'{r[0]}-{r[1]}%': [] for r in ranges}

for column, percentage in null_percentages.items():
    for r in ranges:
        if r[0] <= percentage < r[1]:
            grouped_columns[f'{r[0]}-{r[1]}%'].append(column)

# Imprime los resultados
for range_, columns in grouped_columns.items():
    if columns:
        print(f'Rango de valores nulos: {range_}')
        print(f'Columnas: {columns}')
        print()

Rango de valores nulos: 0-50%
Columnas: ['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M6', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V

In [9]:
range_v = ['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M6', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321']
len(range_v)

220

In [10]:
dataset = dataset[range_v].copy()

In [11]:
dataset.dropna(inplace=True)

## Determinar cuales columnas son categorías o numéricas

In [26]:
data_types = dataset.drop('isFraud',axis=1).dtypes
dataset = dataset[range_v].copy()
num_vars = data_types[data_types != 'object']  # Variables numéricas
cat_vars = data_types[data_types == 'object']  # Variables categóricas

# Obtener nombres de las variables
num_vars_names = num_vars.index.tolist()
cat_vars_names = cat_vars.index.tolist()

In [27]:
# Crear un diccionario para almacenar el número de valores únicos y la categorización para cada columna
dataset.shape
unique_values_info = {
    columna: {
        'Num_Unique_Values': dataset[columna].nunique(),
        'Total_Values': dataset[columna].count(),
        'Is_Categorical': dataset[columna].nunique() / dataset[columna].count() < 0.05
    } for columna in num_vars_names
}

# Convertir el diccionario en un DataFrame para facilitar la ordenación
unique_values_df = pd.DataFrame.from_dict(unique_values_info, orient='index')

# Ordenar el DataFrame por el número de valores únicos
unique_values_df_sorted = unique_values_df.sort_values(by='Num_Unique_Values')

# # Imprimir el resultado
# for index, row in unique_values_df_sorted.iterrows():
#     print(f"Columna: {index}, el número de variables únicas es: {row['Num_Unique_Values']}, "
#           f"Total de valores: {row['Total_Values']}, "
#           f"Podría ser categórica: {row['Is_Categorical']}")

In [14]:
unique_values_df_sorted['Is_Categorical'].value_counts()

Is_Categorical
True     183
False     27
Name: count, dtype: int64

In [41]:
col_num = unique_values_df_sorted[unique_values_df_sorted['Is_Categorical'] == False].index.tolist()

Suponiendo que la cantidad de categorias por columnas no sea mayo al 5% del total de valores dentro de la columnas se obtiene que hay en total 197 posibles columnas categoricas y 13 numéricas

In [28]:
for columna in cat_vars_names :
    print(f"Columna: {columna}, el número de variables es: {dataset[columna].nunique()}")

Columna: ProductCD, el número de variables es: 1
Columna: card4, el número de variables es: 3
Columna: card6, el número de variables es: 2
Columna: P_emaildomain, el número de variables es: 41
Columna: M1, el número de variables es: 1
Columna: M2, el número de variables es: 2
Columna: M3, el número de variables es: 2
Columna: M4, el número de variables es: 3
Columna: M6, el número de variables es: 2


## chi-cuadrado

In [16]:
col_categ_num = unique_values_df_sorted[unique_values_df_sorted['Is_Categorical'] == True].index.tolist()


In [17]:
X = dataset.drop('isFraud', axis=1)
y = dataset['isFraud']

In [29]:

chi2_selector = SelectKBest(chi2, k=10)
X_kbest = chi2_selector.fit_transform(X[col_categ_num], y)

# Obtener las características seleccionadas
print('Características seleccionadas:', X[col_categ_num].columns[chi2_selector.get_support()])

Características seleccionadas: Index(['V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
       'V320'],
      dtype='object')


In [19]:
def one_hot_encode(df, columns):
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(df[columns])
    encoded_df = pd.DataFrame(encoded.toarray(),
                              columns=encoder.get_feature_names_out(columns),
                              index=df.index)
    df = df.drop(columns, axis=1)
    return pd.concat([encoded_df], axis=1)

def binary_encode(df, columns):
    encoder = ce.BinaryEncoder(cols=columns)
    encoded = encoder.fit_transform(df[columns])
    df = df.drop(columns, axis=1)
    return pd.concat([ encoded], axis=1)

# Codificación One-Hot para ProductCD, card4 y card6
X_one_hot = one_hot_encode(X, ['ProductCD', 'card4', 'card6','M1', 'M2', 'M3', 'M4', 'M6'])

# Codificación BinaryEncoder para P_emaildomain
X_binary = binary_encode(X, ['P_emaildomain'])

X_encoded = pd.concat([X_one_hot, X_binary], axis=1)


In [20]:

chi2_selector = SelectKBest(chi2, k=10)
X_kbest = chi2_selector.fit_transform(X_encoded, y)

# Obtener las características seleccionadas
print('Características seleccionadas:', X_encoded.columns[chi2_selector.get_support()])

Características seleccionadas: Index(['card4_discover', 'card6_credit', 'card6_debit', 'M2_F', 'M3_F', 'M3_T',
       'M6_F', 'M6_T', 'P_emaildomain_0', 'P_emaildomain_2'],
      dtype='object')


usando el metodo chi2 para seleccionar las características más relevante son:
 - 'V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
'V320'

-  'card4_discover', 'card6_credit', 'card6_debit', 'M2_F', 'M3_F', 'M3_T',M6_F', 'M6_T', 'P_emaildomain_0', 'P_emaildomain_2'

## Anova

In [46]:
from sklearn.feature_selection import SelectKBest, f_classif

# Selección de las 10 mejores características usando ANOVA
anova_selector = SelectKBest(f_classif, k=10)
X_kbest = anova_selector.fit_transform(X[col_num], y)
lista_num = X[col_num].columns[anova_selector.get_support()].tolist()
print("Características seleccionadas:", X[col_num].columns[anova_selector.get_support()])


Características seleccionadas: Index(['V313', 'V134', 'V318', 'V315', 'V306', 'D2', 'V317', 'V128', 'V308',
       'V307'],
      dtype='object')


In [47]:
for columna in lista_num:
    print(f"Columna: {columna}, el número de variables es: {dataset[columna].nunique()}")

Columna: V313, el número de variables es: 378
Columna: V134, el número de variables es: 446
Columna: V318, el número de variables es: 555
Columna: V315, el número de variables es: 574
Columna: V306, el número de variables es: 614
Columna: D2, el número de variables es: 630
Columna: V317, el número de variables es: 791
Columna: V128, el número de variables es: 1011
Columna: V308, el número de variables es: 1257
Columna: V307, el número de variables es: 2191


- Las columnas númericas relevante de las numpericas son:
    - 'V313', 'V134', 'V318', 'V315', 'V306', 'D2', 'V317', 'V128', 'V308',
       'V307''

## Conclusion

- se uso el metodo chi2 para seleccionar las categorías más relevantes:
 - 'V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
'V320'
 - 'card4_discover', 'card6_credit', 'card6_debit', 'M2_F', 'M3_F', 'M3_T',M6_F', 'M6_T', 'P_emaildomain_0', 'P_emaildomain_2'
- El metodo anova para las categorias numericas.
  'V313', 'V134', 'V318', 'V315', 'V306', 'D2', 'V317', 'V128', 'V308',
'V307'